Họ tên: Nguyễn Ngọc Phước

MSSV: 19127519

# HW3: Các loại bộ nhớ trong CUDA

Với các GPU tương đối mới thì để biên dịch chỉ cần dùng câu lệnh: \
`nvcc tên-file.cu -o tên-file-chạy`

Nhưng trên Colab mình thường lấy được GPU khá cũ là Tesla K80 với compute capability (phiên bản phần cứng) là 3.7; để biên dịch đúng với GPU khá cũ này thì bạn cần dùng câu lệnh: \
`nvcc -arch=sm_37 tên-file.cu -o tên-file-chạy` \
Trong đó, 37 chính là compute capability của GPU Tesla K80.

Để phòng trường hợp khi làm bài bạn lấy được GPU có compute capability x.x nhưng khi chấm bài Thầy lại lấy được GPU có compute capability khác x.x, dưới đây mình sẽ có đoạn code Python để tự động lấy 2 con số ứng với compute capability của GPU và lưu vào 2 biến `major` và `minor`:


In [11]:
from numba import cuda
major, minor = cuda.get_current_device().compute_capability
print(f'GPU compute capability: {major}.{minor}')

GPU compute capability: 7.5


Một khi đã chạy đoạn code Python ở trên, để biên dịch thì bạn sẽ dùng câu lệnh: \
`nvcc -arch=sm_{major}{minor} tên-file.cu -o tên-file-chạy`

Dưới đây, khi làm bài thì bạn có thể tùy ý thêm/xóa cell. Đừng xóa mấy cell có chữ của Thầy là được.

In [12]:
!nvcc HW3.cu -o HW3

In [13]:
!./HW3 in.pnm out.pnm 32 32

**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15843721216 bytes
CMEM: 65536 bytes
L2 cache: 4194304 bytes
SMEM / one SM: 65536 bytes
****************************

Image size (width x height): 512 x 512

Kernel 1, block size 32x32, grid size 16x16
Kernel time: 0.344704 ms
Error: 0.000703

Kernel 2, block size 32x32, grid size 16x16
Kernel time: 0.331808 ms
Error: 0.050429

Kernel 3, block size 32x32, grid size 16x16
Kernel time: 0.246752 ms
Error: 0.258788


In [14]:
!./HW3 in.pnm out.pnm 32 16

**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15843721216 bytes
CMEM: 65536 bytes
L2 cache: 4194304 bytes
SMEM / one SM: 65536 bytes
****************************

Image size (width x height): 512 x 512

Kernel 1, block size 32x16, grid size 16x32
Kernel time: 0.316544 ms
Error: 0.000703

Kernel 2, block size 32x16, grid size 16x32
Kernel time: 0.298240 ms
Error: 0.306432

Kernel 3, block size 32x16, grid size 16x32
Kernel time: 0.225376 ms
Error: 0.450555


Qua kết quả trên, ta có thể thấy thời gian thực hiện các hàm kernel có sự chênh lệch rõ rệt, đặc biệt giữa kernel 1 và 3.

Lý do cho chuyện này là việc sử dụng các vùng nhớ như SMEM, CMEM.

- Khi sử dụng SMEM, thay vì phải truy xuất trực tiếp GMEM, thời gian truy xuất lâu, ta copy vùng GMEM này vào trong SMEM của từng block và thao tác trên đó. Lúc này việc truy xuất khi sử dụng SMEM sẽ nhanh hơn so với GMEM. Kết quả ta thấy được sự chênh lệch về thời gian truy xuất dữ liệu.

- Đối với kernel 3, bình thường ta sử dụng vùng nhớ của filter vẫn là trên GMEM, thay vì vậy, ta copy filter vào CMEM. CMEM cache có tốc độ đọc ghi cao hơn so với GMEM, nên ta thấy được sự cải thiện về thời gian truy xuất. Cũng bởi vì filter là dữ liệu const, không đổi trong quá trình chạy chương trình, cũng như việc truy xuất đến vùng nhớ này có tần suất cao, ta thấy việc copy sang CMEM là hợp lý.